In [1]:
import os
import sys
from pathlib import Path 

In [2]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Mlops\\Malicious_QR_Code_Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\fifty\\OneDrive\\Desktop\\AI - Data Science\\Mlops\\Malicious_QR_Code_Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    csv_dir: Path
    csv_file: str

In [6]:
from src.MaliciousQRCodeDetection.constants import *
from src.MaliciousQRCodeDetection.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH):
        
        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            csv_dir = config.csv_dir,
            csv_file = config.csv_file
        )

        return data_ingestion_config
    

In [8]:
from src.MaliciousQRCodeDetection.logging.logger import logger
from src.MaliciousQRCodeDetection.exception import MaliciousQRException
from src.MaliciousQRCodeDetection.utils.common import get_size
import urllib.request as request

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.csv_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename=self.config.csv_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    logger.error(f'Unexpected error occured while downloading file: {e}')
    raise MaliciousQRException(e,sys)
    

[ 2024-12-23 13:54:09,035] 17 root - INFO - yaml file: config\config.yaml loaded successfully
[ 2024-12-23 13:54:09,036] 31 root - INFO - Created directory at: artifacts
[ 2024-12-23 13:54:09,036] 31 root - INFO - Created directory at: artifacts/data_ingestion
[ 2024-12-23 13:54:11,558] 11 root - INFO - artifacts/data_ingestion/balanced_urls.csv downloaded! with following info: 
Server: GitHub.com
Date: Mon, 23 Dec 2024 08:09:12 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"4adf447b98fc59496d83d9878099b235"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; child-src github.com/assets-cdn/worker/ github.com/webpack/ github.com/assets/